In [2]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
import astrodendro
import os
from astropy import wcs
from astrodendro import Dendrogram, pp_catalog
from astrodendro.analysis import PPStatistic
from astropy.table import Table, hstack, Column
from astropy.utils.console import ProgressBar
import reproject
import pyregion
print(np.__version__)
import aplpy
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

1.16.5


In [49]:
distance        = 8150. # distance to GC; Reid et al. 2014
#Temp            = 20.0
Wave            = (3.0e8/226.0e9)
Wave0           = 1.3e-3
k0              = 0.899
nu              = 3.e08/Wave
nu0             = 3.e08/Wave0
beta            = 1.75
Kappag2d        = k0*((nu/nu0)**beta)
g2d             = 100.0
Kappa           = Kappag2d / g2d
mu              = 2.8 # express everything in H2

dustTemp = 20

# Constants
G = 6.67408e-11
msun = 1.989e33
mh = 1.6737236e-27
pc2cm = 3.08567758e18
as2persr = 4.25e10 
percm2perm = 1.0e6
JyperSr_to_JyperPix = (3.0462*(10**-4))*((0.00013888888888)**2)
JyperSr_to_Jyperassqr = ((np.pi*180.)**(-2))*(3600**(-1))
hplanck = 6.63e-34
clight = 2.99792e8
kboltzmann = 1.381e-23
sin1yr = 3.15569e7
arcsec2pc = distance/((360./(2.*np.pi))*60.*60.)

In [5]:
#################################
### load in column density map
#################################
path = os.path.expanduser('/Users/hph/cmzoom_catalog/catalog_acc/')
column_file = os.path.join(path, 'column_properunits_conv36_source_only.fits')
column_fh = fits.open(column_file)
N_data = column_fh[0].data
print(N_data.shape)

(834, 2226)


In [101]:
#################################
### get pixel size in cm^2
#################################
pixw,pixh = column_fh[0].header['CDELT1'],column_fh[0].header['CDELT2']
pixw_pc,pixh_pc = -1*8150*(pixw*2.*np.pi/360.),8150*(pixh*2.*np.pi/360.)
pixw_cm,pixh_cm = 3.086e+18*pixw_pc,3.086e+18*pixw_pc
print(pixw_pc,pixh_pc)
print(pixw_cm,pixh_cm)
pix_area_cm2 = pixw_cm*pixh_cm

0.4544093802141791 0.45481801242991077
1.4023073473409567e+18 1.4023073473409567e+18


In [108]:
########################################
### set glon bounds in terms of pixels
########################################
colwcs = wcs.WCS(column_fh[0].header)
glonradius = 1.109
glon_low = int(np.rint(colwcs.wcs_world2pix(glonradius, 0, 0)[0]))
glon_high = int(np.rint(colwcs.wcs_world2pix(-glonradius, 0, 0)[0]))
glatmin,glatmax = -0.6,0.6
radius_pc = (8150*(glonradius*2.*np.pi/360.))
area_pc2 = (glatmax-glatmin)*2*np.pi*radius_pc + 2*np.pi*radius_pc**2
print(glon_low,glon_high)
print(area_pc2)

765 1460
157544.8094034682


In [109]:
###########################################
### choose N density cutoff and sum pixels
###########################################
cutoff = 0#1e22
submap = N_data[:,glon_low:glon_high]
submap[np.isnan(submap)]=0
cond = submap>cutoff
condpix = submap[cond] ### select only the matching pixels
particlenum = np.sum(condpix)*pix_area_cm2 ### find matching area in cm^2 and multiply by
print(particlenum)

6.925561966721317e+63


In [110]:
#################################
### convert to CMZ mass
#################################
msun_per_particle = mu*mh/1.989e30
cmzmass_msun = particlenum*msun_per_particle
print("The mass of gas above %1.2e cm^-2 within the innermost %1.3f deg is %2.2e msun"\
      %(cutoff,glonradius,cmzmass_msun))

The mass of gas above 0.00e+00 cm^-2 within the innermost 1.109 deg is 1.63e+07 msun


In [114]:
###########################################
### find relevant angular distance in cmz
###########################################
print(500.*360./(8150*2*np.pi))


3.5150784977351117


In [112]:
##########################################
### convert to CMZ C&D surface density
##########################################
sigma = cmzmass_msun/area_pc2
print("The surface density of gas above %1.2e cm^-2 within the innermost %1.3f deg is %2.2e msun pc^-2"\
      %(cutoff,glonradius,sigma))



The surface density of gas above 0.00e+00 cm^-2 within the innermost 1.109 deg is 1.04e+02 msun pc^-2
